In [ ]:
from googleapiclient.discovery import build
from datetime import datetime
import pandas as pd

# Your API Key
API_KEY = "yourapi"

# Initialize the YouTube API client
youtube = build('youtube', 'v3', developerKey=API_KEY)

# Search for videos with the keyword "XXX"
keyword = "XXX"

# Define the time range, using only the date part
start_date = datetime(2018, 7, 1).date().strftime('%Y-%m-%d')  # From July 1, 2018
end_date = datetime(2024, 7, 1).date().strftime('%Y-%m-%d')  # To July 1, 2024

# Perform the search request, sorted by publication date (oldest to newest), up to 1000 results
search_request = youtube.search().list(
    q=keyword,
    part='snippet',
    type='video',
    publishedAfter=start_date + 'T00:00:00Z',  # Add time part
    publishedBefore=end_date + 'T23:59:59Z',   # Add time part
    maxResults=1000,  # Retrieve up to 1000 results
    order='date',  # Sort by publication date
)

# Execute the first page request
search_response = search_request.execute()

# Prepare data for export to Excel
video_data = []
while search_response:
    for item in search_response['items']:
        video_title = item['snippet']['title']
        video_publish_date = item['snippet']['publishedAt'].split('T')[0]  # Take only the date part
        video_url = f"https://www.youtube.com/watch?v={item['id']['videoId']}"

        video_data.append({
            'Title': video_title,
            'Publish Date': video_publish_date,
            'URL': video_url
        })

    # Check if there's another page of results
    if 'nextPageToken' in search_response:
        next_page_token = search_response['nextPageToken']
        search_response = youtube.search().list(
            q=keyword,
            part='snippet',
            type='video',
            publishedAfter=start_date + 'T00:00:00Z',
            publishedBefore=end_date + 'T23:59:59Z',
            maxResults=500,
            order='date',
            pageToken=next_page_token
        ).execute()
    else:
        break

# Convert to DataFrame
df = pd.DataFrame(video_data)

# Sort DataFrame by publish date (oldest to newest)
df = df.sort_values(by='Publish Date').reset_index(drop=True)

# Save to Excel
file_name = '/Users/your.xlsx'
df.to_excel(file_name, index=False)

print(f"Data has been sorted by publish date and saved to {file_name}")
